In [190]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter

In [203]:
# import our MICROARRAY dataset
GE = pd.read_csv('DATA/GSE54878_NP23_WT.csv')[2:]

#### NA'S #####
# Some of the rows of the data are not annotated to genes... 
print "There are {} entries with missing names".format(sum(GE['Gene.symbol'] != GE['Gene.symbol']))
# count before cleanup 
n_before_drop = len(GE)
# drop na's 
GE = GE.dropna()
# count after cleanup 
n_after_drop = len(GE)

###### REMOVE DUPLICATES 
# count duplicates
counter = Counter(np.array(GE['Gene.symbol']))
duplicates = np.array([(c, k) for (k, c) in counter.items() if c > 1])
print "There are {} entries with duplicated gene symbols".format(len(duplicates))
# drop duplicates 
GE.drop_duplicates(subset = 'Gene.symbol', keep = False, inplace = True)
# count number of entries 
print len(GE)
# count duplicates
counter = Counter(np.array(GE['Gene.symbol']))
duplicates = [(c, k) for (k, c) in counter.items() if c > 1]
print "There are {} duplicated entries after dropout".format(len(duplicates))


# transpose our matrix to have n rows (nb samples), m features (nb genes) 
GE = GE.T
# rename column names with first row (gene symbol)
GE.columns = GE.iloc[0,:]
# drop name columns 
GE = GE[2:]

There are 907 entries with missing names
There are 4954 entries with duplicated gene symbols
12172
There are 0 duplicated entries after dropout


In [192]:
GE['Cdt'] = [x.split()[1] for x in np.array(GE.index)]
TALL = GE[GE['Cdt'] == 'TALL']
TALL = TALL[TALL.columns.difference(['Cdt'])].apply(pd.to_numeric, 0)
WT = GE[GE['Cdt'] == 'WT']
WT = WT[WT.columns.difference(['Cdt'])].apply(pd.to_numeric, 0)
PLK = GE[GE['Cdt'] == 'PLK'][PLK.columns.difference(['Cdt'])]
PLK = PLK.apply(pd.to_numeric, 0)

In [200]:
print TALL.shape
print WT.shape
print PLK.shape

(8, 12172)
(3, 12172)
(3, 12172)


In [194]:
##### DEGs with Fold change thresh = 1.5 #####
##### PLK vs WT  ###########
C1 = pd.DataFrame(PLK.apply(np.mean, 0), columns = ['meanPLK'])
C2 = pd.DataFrame(WT.apply(np.mean, 0), columns = ['meanWT'])
C3 = pd.DataFrame(TALL.apply(np.mean, 0), columns = ['meanTALL'])


In [215]:
PLK_vs_WT = C1.join(C2)
PLK_vs_WT['FC'] = PLK_vs_WT['meanPLK'] - PLK_vs_WT['meanWT']

TALL_vs_WT = C3.join(C2)
TALL_vs_WT['FC'] = TALL_vs_WT['meanTALL'] - TALL_vs_WT['meanWT']

TALL_vs_PLK = C3.join(C1)
TALL_vs_PLK['FC'] = TALL_vs_PLK['meanTALL'] - TALL_vs_PLK['meanPLK']

In [218]:
epsilon = 0.5
tresh = 1.5
#PLK_vs_WT[((PLK_vs_WT['FC'] > (1  + epsilon)) | (PLK_vs_WT['FC'] < (1 - epsilon)))]
PLK_vs_WT[(np.abs(PLK_vs_WT['FC'] > tresh ))]


,meanPLK,meanWT,FC
Gene.symbol,,,
Fgfbp1,9.071333,7.468333,1.603000
Hoxa5,12.286000,8.827667,3.458333
Hoxa7,11.074333,8.264000,2.810333
Insl6,13.033000,10.291667,2.741333
Meis1,11.013333,9.284000,1.729333
Myl10,12.668000,8.920000,3.748000
Slc5a9,10.937333,8.449333,2.488000


In [219]:
# TALL_vs_WT[((TALL_vs_WT['FC'] > (1  + epsilon)) | (TALL_vs_WT['FC'] < (1 - epsilon)))].sort_values('FC')
TALL_vs_WT[(np.abs(TALL_vs_WT['FC'] > tresh ))]

,meanTALL,meanWT,FC
Gene.symbol,,,
4930550C14Rik,9.842750,8.216667,1.626083
Aldh1b1,10.106625,7.806000,2.300625
Arg1,9.791750,7.584333,2.207417
Bcas1,9.761500,7.778000,1.983500
Bmp7,9.808000,7.903333,1.904667
Camk2b,9.767625,8.167667,1.599958
Ccl8,11.244000,8.708000,2.536000
Ctsg,9.081000,7.548000,1.533000
Ddc,9.199375,7.576000,1.623375


In [220]:
# TALL_vs_PLK[((TALL_vs_PLK['FC'] > (1  + epsilon)) | (TALL_vs_PLK['FC'] < (1 - epsilon)))].sort_values('FC')
TALL_vs_PLK[(np.abs(TALL_vs_PLK['FC'] > tresh ))]

,meanTALL,meanPLK,FC
Gene.symbol,,,
Aldh1b1,10.106625,8.340333,1.766292
Arg1,9.791750,7.458000,2.333750
Bcas1,9.761500,8.201000,1.560500
Ccl8,11.244000,9.224000,2.020000
Dlx1,9.145500,7.595000,1.550500
Dtx1,10.102625,8.512667,1.589958
Fcgr4,9.955625,8.160000,1.795625
Gm266,9.448250,7.778667,1.669583
Nfil3,9.963375,8.221333,1.742042
